# Test running python in the background

Let's start by creating the resources we are going to need: a database and a container.

In [3]:
import os
import time
import azure.cosmos

db_name = "mybgtest"
container_name = "myidcltest"

# Create the database if it doesn't exist
try:
    database = cosmos_client.create_database_if_not_exists(db_name)
    print('Database "' + db_name + '" created')
    time.sleep(2)
except azure.cosmos.errors.CosmosHttpResponseError as e:
    if e.status_code == 409:
       print('Database already exists')
    else: 
        raise

# Reset the container
database_client = cosmos_client.get_database_client(db_name)
containers = database_client.list_containers()
if (any(container['id'] == container_name for container in containers)):
    database_client.delete_container(container_name)
    print('Container "' + container_name + '" dropped')
database_client.create_container(container_name, azure.cosmos.PartitionKey(path="/id"))
print('Container "' + container_name + '" created')

Database "mybgtest" created
Container "myidcltest" dropped
Container "myidcltest" created


### Importing documents

We will need some test data to work with, so we import 10,000 documents. Each document contains 2 fields: `field1` which has a random string value and `field2` which has a random integer value:

```
{
  "field1":"Garry75",
  "field2":405
}
```

This is going to take some time, so feel free to get a coffee ☕ in the meantime!

In [5]:
import urllib.request  as urllib2
import json
import uuid


container_client = database_client.get_container_client(container_name)

for i in range(5):
    req = urllib2.Request("http://api.open-notify.org/iss-now.json")
    response = urllib2.urlopen(req)

    obj = json.loads(response.read())
    # {"iss_position": {"longitude": "-151.4464", "latitude": "-51.0588"}, "timestamp": 1579293714, "message": "success"}
    print(obj)
 
    if obj['message'] == 'success':
        new_item = { 'id': str(uuid.uuid4()), 'timestamp': obj['timestamp'], 'latitude': obj['iss_position']['latitude'], 'longitude': obj['iss_position']['latitude'] }
        container_client.create_item(new_item)

        print (obj['timestamp'])
        print (obj['iss_position']['latitude'], obj['iss_position']['latitude'])
        
        time.sleep(5)
    else:
        print ('API failure: ', json.dumps(obj))



{'iss_position': {'longitude': '-33.9161', 'latitude': '27.9685'}, 'timestamp': 1579295789, 'message': 'success'}
1579295789
27.9685 27.9685
{'iss_position': {'longitude': '-33.6551', 'latitude': '28.2217'}, 'timestamp': 1579295795, 'message': 'success'}
1579295795
28.2217 28.2217
{'iss_position': {'longitude': '-33.4166', 'latitude': '28.4515'}, 'timestamp': 1579295800, 'message': 'success'}
1579295800
28.4515 28.4515
{'iss_position': {'longitude': '-33.1769', 'latitude': '28.6807'}, 'timestamp': 1579295805, 'message': 'success'}
1579295805
28.6807 28.6807
{'iss_position': {'longitude': '-32.9120', 'latitude': '28.9323'}, 'timestamp': 1579295810, 'message': 'success'}
1579295810
28.9323 28.9323


### Testing a first query

Now that we have our test data, time to issue some queries! We start by fetching all documents with a `field1` value of `Mario86`:

In [6]:
query = 'SELECT * FROM c'
start = time.time()
results = list(container_client.query_items(
    query=query,
    enable_cross_partition_query=True))
end = time.time()

print('Got ' + str(len(results)) + ' result(s)')
print('Time elapsed: %d ms'% ((end - start) * 1000))
print('Request charge: ' + container_client.client_connection.last_response_headers['x-ms-request-charge'] + ' RUs')

Got 10 result(s)
Time elapsed: 71 ms
Request charge: 2.53 RUs


In [6]:
cosmos_client.delete_database(db_name)